# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [12]:
from typing import Tuple, List
import random
from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove

Wielkość planszy

In [13]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [40]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game

    def how_many_in(self, ee):
        return len([i for i in ee if i is not None])

    def evaluate_position(self, is_maximizing_player)->float:
        if self.game.is_finished():
            if is_maximizing_player:
                return -10000
            elif not is_maximizing_player:
                return 10000
            else:
                return 0
        elif not self.game.is_finished():
            fields = self.game.state.fields
            value = 0
            p1 = self.game.state._other_player.char
            p2 = self.game.state._current_player.char
            for column_id in range(len(fields)):  # verticals
                for start_row_id in range(len(fields[column_id]) - 3):
                    vertical = [fields[column_id][start_row_id + i] for i in range(4)]
                    vertical = [i.char if i is not None else None for i in vertical]
                    if p1 in vertical and p2 not in vertical:
                        value += self.how_many_in(vertical)
                    elif p2 in vertical and p1 not in vertical:
                        value -= self.how_many_in(vertical)

            for start_column_id in range(len(fields) - 3):  # horizontals
                for row_id in range(len(fields[start_column_id])):
                    horizontal = [fields[start_column_id + i][row_id] for i in range(4)]
                    horizontal = [i.char if i is not None else None for i in horizontal]
                    if p1 in horizontal and p2 not in horizontal:
                        value += self.how_many_in(horizontal)
                    elif p2 in horizontal and p1 not in horizontal:
                        value -= self.how_many_in(horizontal)

            for start_column_id in range(len(fields) - 3):  # diagonals
                for start_row_id in range(len(fields[start_column_id]) - 3):
                    diag1 = [fields[start_column_id + i][start_row_id + i] for i in range(4)]
                    diag1 = [i.char if i is not None else None for i in diag1]
                    diag2 = [fields[start_column_id + i][ROW_COUNT - 1 - start_row_id - i] for i in range(4)]
                    diag2 = [i.char if i is not None else None for i in diag2]
                    if p1 in diag1 and p2 not in diag1:
                        value += self.how_many_in(diag1)
                    elif p2 in diag1 and p1 not in diag1:
                        value -= self.how_many_in(diag1)
                    if p1 in diag2 and p2 not in diag2:
                        value += self.how_many_in(diag2)
                    elif p2 in diag2 and p1 not in diag2:
                        value -= self.how_many_in(diag2)
            return value

    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)->Tuple[float, int]:
        """Returns column index and score"""
        if self.game.is_finished() or depth == 0:
            q = {True: -1, False: 1}[is_maximizing_player]
            y = self.evaluate_position(is_maximizing_player)+depth
            return y, None
        if is_maximizing_player:
            value = -10000
            col = random.choice(self.game.get_moves()).column
            for column in self.get_valid_move():
                self.game.make_move(ConnectFourMove(column))
                new_value = self.minimax(depth-1, alpha, beta, False)[0]
                self.undo_move(column)
                if new_value > value:
                    value = new_value
                    col = column
                if value >= beta:
                    break
                alpha = max(alpha, value)
            return value, col
        else:
            value = 10000
            col = random.choice(self.game.get_moves()).column
            for column in self.get_valid_move():
                self.game.make_move(ConnectFourMove(column))
                new_value = self.minimax(depth-1, alpha, beta, True)[0]
                self.undo_move(column)
                if new_value < value:
                    value = new_value
                    col = column
                if value <= alpha:
                    break
                beta = min(beta, value)
            return value, col


    def get_best_move(self, depth)->int:
        return self.minimax(depth=depth, alpha=-10000, beta=10000, is_maximizing_player=True)

    def get_valid_move(self):
        return [i.column for i in self.game.get_moves()]

    def undo_move(self, move):
        fields = self.game.state.fields[move]
        height = len(fields)
        for i in range(height):
            if fields[height-1-i] is not None:
                self.game.state.fields[move][height-1-i] = None
                self.game.state._current_player, self.game.state._other_player = self.game.state._other_player, self.game.state._current_player
                break


Rozgrywka

In [43]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    z = x.get_best_move(4)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(6)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][x]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][x]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][ ]
[o][x][ ][ ][ ][ ][x]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][ ][ ]
[o][x][ ][ ][ ][ ][x]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[x][ ][ ][ ][ ][ ][ ]
[o][ ][ ][ ][ ][

głębokość 4 vs 6
6 wygrało

In [41]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(5)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(5)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][

5 vs 5
wygrał drugi
większość planszy wypełniona

In [39]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(3)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(5)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][

3 vs 5
wygrała siła 5

In [34]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(5)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(3)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][x][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][o][ ][x][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][o][ ][x][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][o][ ][x][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][

5 vs 3
wygrała głębokość 5

In [35]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(5)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(7)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][x][ ][ ][ ][ ][ ]
[ ][o][ ][ ][ ][

5 vs 7
remis

In [36]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(3)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        break
    print(x.game)
    y = x.get_best_move(7)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][

3 vs 7
remis

In [45]:
p1 = Player("o")
p2 = Player("x")
game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
x = MinMaxSolver(game)
print(x.game)
while not x.game.is_finished():
    # z = int(input())
    z = x.get_best_move(3)
    x.game.make_move(ConnectFourMove(z[1]))
    if x.game.is_finished():
        print(x.game)
        break
    print(x.game)
    y = x.get_best_move(3)
    # print(y)
    x.game.make_move(ConnectFourMove(y[1]))
    print(x.game)

Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
Current player: o
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][

Ogólne wnioski:
n - remis
o\x|_3_|_4_|_5_|_6_|_7_|
  3| o | o | x | o | x |
  4| x | x | x | x | x |
  5| o | o | x | x | n |
  6| o | o | x | x | x |
  7| o | o | x | o | o |

  Dla głębokości mniejszych od 6 szybko znajduje ruch zaś dla większych czas wyszukiwania się wydłuża.
  Większość razy wygrywa ten co ma większą głębokość ale czasem zdarza się że wygrywa ze słabszą. Jednkaże dla takich samych głęłbokości wygrywa czasem ten co zaczyna, a czasem ten co jest drugi.